In [0]:
%pip install xgboost lightgbm


In [0]:
# =========================
# ETAPA 1 -CRIAÇÃO DA TABELA DE PREVISÃO DE PRODUTO E LOJA
# =========================
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, DoubleType, LongType, StringType
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
import pandas as pd
import numpy as np

# =========================
# Carregar dados agregados por produto + loja + mês
# =========================
df_vendas = spark.table("ted_dev.dbt_acroccia_aw_marts.fato_pedidos_2")

# Criar coluna ano_mes e converter para date
df_vendas = df_vendas.withColumn("ano_mes", F.date_format("data_de_venda", "yyyy-MM"))
df_agg = df_vendas.groupBy("produto", "id_loja", "ano_mes") \
                  .agg(F.sum("qtde").alias("qtde"))

df_agg = df_agg.withColumn(
    "ano_mes",
    F.to_date(F.concat_ws("-", F.col("ano_mes"), F.lit("01")), "yyyy-MM-dd")
)

df_agg = df_agg.filter(F.col("id_loja").isNotNull() & (F.col("qtde") > 0))
df_agg = df_agg.repartition("produto", "id_loja")

# =========================
# Função de forecast otimizada
# =========================
def forecast_3meses(pdf):
    produto = pdf['produto'].iloc[0]
    loja = pdf['id_loja'].iloc[0]
    pdf = pdf.sort_values('ano_mes').copy()
    pdf['ano_mes'] = pd.to_datetime(pdf['ano_mes'])
    
    # Se histórico insuficiente ou vendas zeradas: usar média móvel
    if len(pdf) < 3 or pdf['qtde'].sum() == 0:
        ma_pred = pdf['qtde'].mean() * 3 if len(pdf) > 0 else 0
        return pd.DataFrame({
            'produto':[produto],
            'id_loja':[loja],
            'rf':[0],
            'xgb':[0],
            'lgb':[0],
            'ma':[ma_pred]
        })
    
    # Features de tempo
    pdf['ano'] = pdf['ano_mes'].dt.year
    pdf['mes'] = pdf['ano_mes'].dt.month
    X = pdf[['ano','mes']]
    y = pdf['qtde'].values
    
    # Treinar modelos
    rf_model = RandomForestRegressor(n_estimators=5, random_state=42)
    rf_model.fit(X, y)
    xgb_model = xgb.XGBRegressor(n_estimators=5, n_jobs=-1, random_state=42)
    xgb_model.fit(X, y)
    lgb_model = lgb.LGBMRegressor(n_estimators=5, n_jobs=-1, random_state=42)
    lgb_model.fit(X, y)
    
    # Próximos 3 meses
    last_date = pdf['ano_mes'].max()
    future_dates = pd.date_range(start=last_date + pd.offsets.MonthBegin(1), periods=3, freq='MS')
    future_df = pd.DataFrame({'ano': future_dates.year, 'mes': future_dates.month})
    
    # Previsões
    rf_pred = np.clip(rf_model.predict(future_df), 0, None).sum()
    xgb_pred = np.clip(xgb_model.predict(future_df), 0, None).sum()
    lgb_pred = np.clip(lgb_model.predict(future_df), 0, None).sum()
    ma_pred = pdf['qtde'].rolling(3, min_periods=1).mean().iloc[-1] * 3
    
    return pd.DataFrame({
        'produto':[produto],
        'id_loja':[loja],
        'rf':[rf_pred],
        'xgb':[xgb_pred],
        'lgb':[lgb_pred],
        'ma':[ma_pred]
    })

# =========================
# Schema
# =========================
# =========================
# Schema corrigido
# =========================

schema = StructType([
    StructField("produto", StringType()),   # produto como string
    StructField("id_loja", StringType()),   # id_loja como string
    StructField("rf", DoubleType()),
    StructField("xgb", DoubleType()),
    StructField("lgb", DoubleType()),
    StructField("ma", DoubleType())
])


# =========================
# Aplicar forecast em paralelo
# =========================
df_forecast = df_agg.groupBy("produto","id_loja") \
                    .applyInPandas(forecast_3meses, schema=schema)

# =========================
# Mostrar resultado rápido
# =========================
df_forecast.show(5, truncate=False)

# =========================
# Salvar em Delta
# =========================
spark.sql("CREATE SCHEMA IF NOT EXISTS ted_dev.dev_andre_silva")
df_forecast.write.mode("overwrite") \
    .format("delta") \
    .saveAsTable("ted_dev.dev_andre_silva.forecast_loja_3meses")

In [0]:
%sql
-- =========================
-- Exemplo Sazonalidade
-- =========================
SELECT
    data_de_venda,
    SUM(qtde) AS qtde_mensal
FROM ted_dev.dbt_acroccia_aw_marts.fato_pedidos_2
WHERE produto = 'AWC Logo Cap'
  AND id_loja = '1002'
GROUP BY data_de_venda
ORDER BY data_de_venda;


In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def forecast_3meses_com_metrica(pdf):
    produto = pdf['produto'].iloc[0]
    loja = pdf['id_loja'].iloc[0]
    pdf = pdf.sort_values('ano_mes').copy()
    pdf['ano_mes'] = pd.to_datetime(pdf['ano_mes'])

    if len(pdf) < 6 or pdf['qtde'].sum() == 0:
        ma_pred = pdf['qtde'].mean() * 3 if len(pdf) > 0 else 0
        return pd.DataFrame({
            'produto':[produto],
            'id_loja':[loja],
            'rf':[0], 'xgb':[0], 'lgb':[0], 'ma':[ma_pred],
            'rf_mae':[None], 'rf_rmse':[None], 'rf_r2':[None],
            'xgb_mae':[None], 'xgb_rmse':[None], 'xgb_r2':[None],
            'lgb_mae':[None], 'lgb_rmse':[None], 'lgb_r2':[None]
        })

    pdf['ano'] = pdf['ano_mes'].dt.year
    pdf['mes'] = pdf['ano_mes'].dt.month
    X = pdf[['ano','mes']]
    y = pdf['qtde'].values

    # Separar treino e teste (últimos 3 meses como teste)
    X_train, X_test = X[:-3], X[-3:]
    y_train, y_test = y[:-3], y[-3:]

    # Treinar modelos
    rf_model = RandomForestRegressor(n_estimators=5, random_state=42)
    rf_model.fit(X_train, y_train)
    xgb_model = xgb.XGBRegressor(n_estimators=5, n_jobs=-1, random_state=42)
    xgb_model.fit(X_train, y_train)
    lgb_model = lgb.LGBMRegressor(n_estimators=5, n_jobs=-1, random_state=42)
    lgb_model.fit(X_train, y_train)

    # Previsões nos 3 meses de teste
    rf_test_pred = rf_model.predict(X_test)
    xgb_test_pred = xgb_model.predict(X_test)
    lgb_test_pred = lgb_model.predict(X_test)

    # Métricas
    rf_mae = mean_absolute_error(y_test, rf_test_pred)
    rf_rmse = mean_squared_error(y_test, rf_test_pred, squared=False)
    rf_r2 = r2_score(y_test, rf_test_pred)

    xgb_mae = mean_absolute_error(y_test, xgb_test_pred)
    xgb_rmse = mean_squared_error(y_test, xgb_test_pred, squared=False)
    xgb_r2 = r2_score(y_test, xgb_test_pred)

    lgb_mae = mean_absolute_error(y_test, lgb_test_pred)
    lgb_rmse = mean_squared_error(y_test, lgb_test_pred, squared=False)
    lgb_r2 = r2_score(y_test, lgb_test_pred)

    # Previsão para os próximos 3 meses
    last_date = pdf['ano_mes'].max()
    future_dates = pd.date_range(start=last_date + pd.offsets.MonthBegin(1), periods=3, freq='MS')
    future_df = pd.DataFrame({'ano': future_dates.year, 'mes': future_dates.month})

    rf_pred = np.clip(rf_model.predict(future_df), 0, None).sum()
    xgb_pred = np.clip(xgb_model.predict(future_df), 0, None).sum()
    lgb_pred = np.clip(lgb_model.predict(future_df), 0, None).sum()
    ma_pred = pdf['qtde'].rolling(3, min_periods=1).mean().iloc[-1] * 3

    return pd.DataFrame({
        'produto':[produto],
        'id_loja':[loja],
        'rf':[rf_pred], 'xgb':[xgb_pred], 'lgb':[lgb_pred], 'ma':[ma_pred],
        'rf_mae':[rf_mae], 'rf_rmse':[rf_rmse], 'rf_r2':[rf_r2],
        'xgb_mae':[xgb_mae], 'xgb_rmse':[xgb_rmse], 'xgb_r2':[xgb_r2],
        'lgb_mae':[lgb_mae], 'lgb_rmse':[lgb_rmse], 'lgb_r2':[lgb_r2]
    })


In [0]:
# =========================
# ETAPA 2 - ESTUDO DOS MODELOS E ANÁLISE POR MÉTRICA
# =========================
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, DoubleType, StringType
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np

# =========================
# Carregar e preparar dados
# =========================
df_vendas = spark.table("ted_dev.dbt_acroccia_aw_marts.fato_pedidos_2")

df_vendas = df_vendas.withColumn("ano_mes", F.date_format("data_de_venda", "yyyy-MM"))
df_agg = df_vendas.groupBy("produto", "id_loja", "ano_mes") \
                  .agg(F.sum("qtde").alias("qtde"))

df_agg = df_agg.withColumn(
    "ano_mes",
    F.to_date(F.concat_ws("-", F.col("ano_mes"), F.lit("01")), "yyyy-MM-dd")
)

df_agg = df_agg.filter(F.col("id_loja").isNotNull() & (F.col("qtde") > 0))
df_agg = df_agg.repartition("produto", "id_loja")

# =========================
# Função de forecast com métricas
# =========================
def forecast_3meses_com_metrica(pdf):
    produto = pdf['produto'].iloc[0]
    loja = pdf['id_loja'].iloc[0]
    pdf = pdf.sort_values('ano_mes').copy()
    pdf['ano_mes'] = pd.to_datetime(pdf['ano_mes'])

    if len(pdf) < 6 or pdf['qtde'].sum() == 0:
        ma_pred = pdf['qtde'].mean() * 3 if len(pdf) > 0 else 0
        return pd.DataFrame({
            'produto':[produto],
            'id_loja':[loja],
            'rf':[0], 'xgb':[0], 'lgb':[0], 'ma':[ma_pred],
            'rf_mae':[None], 'rf_rmse':[None], 'rf_r2':[None],
            'xgb_mae':[None], 'xgb_rmse':[None], 'xgb_r2':[None],
            'lgb_mae':[None], 'lgb_rmse':[None], 'lgb_r2':[None]
        })

    pdf['ano'] = pdf['ano_mes'].dt.year
    pdf['mes'] = pdf['ano_mes'].dt.month
    X = pdf[['ano','mes']]
    y = pdf['qtde'].values

    X_train, X_test = X[:-3], X[-3:]
    y_train, y_test = y[:-3], y[-3:]

    rf_model = RandomForestRegressor(n_estimators=5, random_state=42)
    rf_model.fit(X_train, y_train)
    xgb_model = xgb.XGBRegressor(n_estimators=5, n_jobs=-1, random_state=42)
    xgb_model.fit(X_train, y_train)
    lgb_model = lgb.LGBMRegressor(n_estimators=5, n_jobs=-1, random_state=42)
    lgb_model.fit(X_train, y_train)

    rf_test_pred = rf_model.predict(X_test)
    xgb_test_pred = xgb_model.predict(X_test)
    lgb_test_pred = lgb_model.predict(X_test)

    rf_mae = mean_absolute_error(y_test, rf_test_pred)
    rf_rmse = mean_squared_error(y_test, rf_test_pred, squared=False)
    rf_r2 = r2_score(y_test, rf_test_pred)

    xgb_mae = mean_absolute_error(y_test, xgb_test_pred)
    xgb_rmse = mean_squared_error(y_test, xgb_test_pred, squared=False)
    xgb_r2 = r2_score(y_test, xgb_test_pred)

    lgb_mae = mean_absolute_error(y_test, lgb_test_pred)
    lgb_rmse = mean_squared_error(y_test, lgb_test_pred, squared=False)
    lgb_r2 = r2_score(y_test, lgb_test_pred)

    last_date = pdf['ano_mes'].max()
    future_dates = pd.date_range(start=last_date + pd.offsets.MonthBegin(1), periods=3, freq='MS')
    future_df = pd.DataFrame({'ano': future_dates.year, 'mes': future_dates.month})

    rf_pred = np.clip(rf_model.predict(future_df), 0, None).sum()
    xgb_pred = np.clip(xgb_model.predict(future_df), 0, None).sum()
    lgb_pred = np.clip(lgb_model.predict(future_df), 0, None).sum()
    ma_pred = pdf['qtde'].rolling(3, min_periods=1).mean().iloc[-1] * 3

    return pd.DataFrame({
        'produto':[produto],
        'id_loja':[loja],
        'rf':[rf_pred], 'xgb':[xgb_pred], 'lgb':[lgb_pred], 'ma':[ma_pred],
        'rf_mae':[rf_mae], 'rf_rmse':[rf_rmse], 'rf_r2':[rf_r2],
        'xgb_mae':[xgb_mae], 'xgb_rmse':[xgb_rmse], 'xgb_r2':[xgb_r2],
        'lgb_mae':[lgb_mae], 'lgb_rmse':[lgb_rmse], 'lgb_r2':[lgb_r2]
    })

# =========================
# Schema
# =========================
schema = StructType([
    StructField("produto", StringType()),
    StructField("id_loja", StringType()),
    StructField("rf", DoubleType()),
    StructField("xgb", DoubleType()),
    StructField("lgb", DoubleType()),
    StructField("ma", DoubleType()),
    StructField("rf_mae", DoubleType()),
    StructField("rf_rmse", DoubleType()),
    StructField("rf_r2", DoubleType()),
    StructField("xgb_mae", DoubleType()),
    StructField("xgb_rmse", DoubleType()),
    StructField("xgb_r2", DoubleType()),
    StructField("lgb_mae", DoubleType()),
    StructField("lgb_rmse", DoubleType()),
    StructField("lgb_r2", DoubleType())
])

# =========================
# Aplicar forecast e mostrar resultado
# =========================
df_forecast = df_agg.groupBy("produto","id_loja") \
                    .applyInPandas(forecast_3meses_com_metrica, schema=schema)

df_forecast.show(truncate=False)

In [0]:
%python
pip install prophet

In [0]:
%python
# ============================================
# ETAPA 3 - Carregar a tabela fato
# ============================================
import pandas as pd
from prophet import Prophet

# Ler tabela do Databricks (Spark → Pandas)
df = spark.table("ted_dev.dbt_acroccia_aw_marts.fato_pedidos_2").toPandas()

# Converter datas
df["data_de_venda"] = pd.to_datetime(df["data_de_venda"])

print(df.head())

In [0]:
%python

# ============================================
# ETAPA 3 - Crescimento por Grupo (Províncias EUA x Países resto do mundo)
# ============================================
from prophet import Prophet

# Criar coluna grupo (Provincia = EUA, Pais = resto do mundo)
df["grupo"] = df["pais"].apply(lambda x: "Provincia" if x == "United States" else "Pais")

# Agregar mensalmente por grupo
df_group = df.groupby(
    [df["data_de_venda"].dt.to_period("M"), "grupo"]
)["qtde"].sum().reset_index()

df_group["ano_mes"] = df_group["data_de_venda"].dt.to_timestamp()

# Função para rodar Prophet em cada grupo
def forecast_group(grupo_nome):
    serie = df_group[df_group["grupo"] == grupo_nome][["ano_mes", "qtde"]].rename(
        columns={"ano_mes": "ds", "qtde": "y"}
    )

    m = Prophet(yearly_seasonality=True, daily_seasonality=False)
    m.fit(serie)

    future = m.make_future_dataframe(periods=3, freq="M")
    forecast = m.predict(future)

    # Previsão total dos próximos 3 meses
    total_previsto = forecast.tail(3)["yhat"].sum()
    return total_previsto, forecast, m

# Rodar para "Provincia" (EUA) e "Pais" (resto do mundo)
prev_provincia, forecast_provincia, m_provincia = forecast_group("Provincia")
prev_pais, forecast_pais, m_pais = forecast_group("Pais")

print("Demanda prevista - Próximos 3 meses:")
print(f"Provincias (EUA): {prev_provincia:.0f}")
print(f"Países (Resto do mundo): {prev_pais:.0f}")

# Comparar quem cresceu mais
if prev_provincia > prev_pais:
    print("👉 O grupo Provincias (EUA) apresentou mais crescimento.")
else:
    print("👉 O grupo Países (resto do mundo) apresentou mais crescimento.")

# (Opcional) Gráficos
fig1 = m_provincia.plot(forecast_provincia)
fig2 = m_pais.plot(forecast_pais)


In [0]:
%python

# ============================================
# ETAPA 4 - Estimativa de zíperes (Gloves)
# ============================================
import pandas as pd
from prophet import Prophet

# Ler tabela do Databricks (Spark → Pandas)
df = spark.table("ted_dev.dbt_acroccia_aw_marts.fato_pedidos_2").toPandas()

# Converter datas
df["data_de_venda"] = pd.to_datetime(df["data_de_venda"])

# Filtrar apenas luvas
df_gloves = df[df["produto"].str.contains("Gloves", case=False, na=False)]

# Agregar mensalmente
serie_gloves = df_gloves.groupby(
    df_gloves["data_de_venda"].dt.to_period("M")
)["qtde"].sum().reset_index()

serie_gloves["ano_mes"] = serie_gloves["data_de_venda"].dt.to_timestamp()
serie_gloves = serie_gloves.rename(columns={"ano_mes": "ds", "qtde": "y"})

# Ajustar modelo Prophet
m_gloves = Prophet(yearly_seasonality=True)
m_gloves.fit(serie_gloves)

# Previsão 3 meses
future_gloves = m_gloves.make_future_dataframe(periods=3, freq="M")
forecast_gloves = m_gloves.predict(future_gloves)

# Calcular total previsto
prev_gloves = forecast_gloves.tail(3)["yhat"].sum()

# Estimativa de zíperes (2 por par)
zippers_needed = int(prev_gloves * 2)
print("Zíperes necessários (próximos 3 meses):", zippers_needed)

In [0]:
%sql
WITH previsao_luvas AS (
    SELECT 
        produto,
        id_loja,
        rf, xgb, lgb, ma,
        (rf + xgb + lgb + ma)/4 AS media_modelos
    FROM ted_dev.dev_andre_silva.forecast_loja_3meses 
    WHERE LOWER(produto) LIKE '%gloves%'
)
SELECT
    SUM(media_modelos) * 2 AS ziperes_necessarios_prox_3_meses
FROM previsao_luvas;


In [0]:
%sql
    SELECT 
        produto,
        id_loja,
        rf, xgb, lgb, ma,
        (rf + xgb + lgb + ma)/4 AS media_modelos
    FROM ted_dev.dev_andre_silva.forecast_loja_3meses 
    WHERE LOWER(produto) LIKE '%gloves%'


In [0]:
%sql
SELECT
    YEAR(data_de_venda) AS ano,
    MONTH(data_de_venda) AS mes,
    SUM(qtde) AS total_luvas_vendidas
FROM ted_dev.dbt_acroccia_aw_marts.fato_pedidos_2
WHERE LOWER(produto) LIKE '%glove%'
GROUP BY YEAR(data_de_venda), MONTH(data_de_venda)
ORDER BY ano, mes;



